In [1]:
import numpy as np
import pandas as pd
import mne
import os

In [2]:
root_path='/mnt/ssd_4tb_0/data/tuhab_preprocessed'
csv_path='/mnt/ssd_4tb_0/data/tuhab_preprocessed/file_lengths_map.csv'

In [3]:
ex_data_filename = os.listdir(root_path)[0]
ex_data_filename

'aaaaalgi_s001_t001_preprocessed.npy'

In [4]:
ex_data = np.load(os.path.join(root_path, ex_data_filename))
ex_data.shape

(86300, 19)

In [5]:
# from feature_calc import 

In [6]:
import numpy as np
import mne

data = ex_data.T
sfreq = 100.
ch_names = [f'EEG {i+1}' for i in range(data.shape[0])]
ch_types = ['eeg'] * len(ch_names)

info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
raw = mne.io.RawArray(data, info)


Creating RawArray with float64 data, n_channels=19, n_times=86300
    Range : 0 ... 86299 =      0.000 ...   862.990 secs
Ready.


In [7]:
epochs = mne.make_fixed_length_epochs(raw, duration=10.0, preload=True)

Not setting metadata
86 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 86 events and 1000 original time points ...
0 bad epochs dropped


In [8]:
psds, freqs = epochs.compute_psd(fmin=0.5, fmax=35.0, method='multitaper').get_data(return_freqs=True)

    Using multitaper spectrum estimation with 7 DPSS windows


In [9]:
psds = psds / np.sum(psds, axis=-1, keepdims=True)  # (n_epochs, n_channels, n_freqs)

In [10]:
# alpha_idx = np.logical_and(freqs >= 0.5, freqs <= 4)
alpha_idx = np.logical_and(freqs >= 8, freqs <= 13)
alpha_power = psds[:, :, alpha_idx].mean(axis=-1)  # (n_epochs, n_channels)

In [12]:
alpha_power[0]

array([0.00022829, 0.00020346, 0.00023693, 0.00021627, 0.00044502,
       0.00026892, 0.00018058, 0.00015   , 0.00025559, 0.00083293,
       0.00026497, 0.00028399, 0.00026672, 0.00019304, 0.00059648,
       0.00040383, 0.00024965, 0.00037948, 0.0002121 ])

In [2]:
import pickle

dict_path = 'preprocessing/outputs/alpha_power_dict_dict.pkl'

with open(dict_path, 'rb') as f:
    alpha_dict = pickle.load(f)

alpha_dict['aaaaalgi_s001_t001']

EOFError: Ran out of input